In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.1          ✔ dplyr   0.8.5     
✔ tidyr   1.0.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "a6-essemble"

In [2]:
data <- read.csv("../input/a6-essemble/Earthquate_Damage.csv")
cat_cols <- c("land_surface_condition","foundation_type",
              "roof_type","ground_floor_type",
             "other_floor_type","position",
              "plan_configuration","legal_ownership_status")
install.packages("nnet")
library(nnet)
j <- 1
for(i in 1:length(cat_cols)){
    tmp <- class.ind(data[,cat_cols[i]])
    colnames(tmp) <- (j:(j+length(colnames(tmp)) - 1))
    data <- cbind(data,tmp)
    j <- (j+length(colnames(tmp)))
}
data <- data[,!(names(data)%in%cat_cols)]
scaled_data <- scale(data[,1:31],center=TRUE,scale=TRUE)
other_data <- data[,32:70]
data <- cbind(scaled_data,other_data)
rm(scaled_data)
rm(other_data)
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
rm(trn_idx)
rm(tmp)
rm(val_idx)
perf_eval_multi <- function(cm){
  
  # Simple Accuracy
  ACC = sum(diag(cm))/sum(cm)
  
  # Balanced Correction Rate
  BCR = 1
  for (i in 1:dim(cm)[1]){
    BCR = BCR*(cm[i,i]/sum(cm[i,])) 
  }
  
  BCR = BCR^(1/dim(cm)[1])
  
  return(c(ACC, BCR))
}
prediction_one <- function(pred){
    tmp <- apply(pred,1,which.max)
    for (i in 1:nrow(pred)){
        pred[i,tmp[i]] <- 1
        pred[i,-tmp[i]] <- 0
    }
    return(pred)
}
library(party)
all_train <- rbind(train,val)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: grid

Loading required package: mvtnorm

Loading required package: modeltools

Loading required package: stats4

Loading required package: strucchange

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich


Attaching package: ‘strucchange’


The following object is masked from ‘package:stringr’:

    boundary




In [3]:
library(randomForest)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin




In [4]:
attach(all_train)

In [5]:
all_train$damage_grade <- as.factor(all_train$damage_grade)
test$damage_grade <- as.factor(test$damage_grade)

In [6]:
rf_result = matrix(0,10,3)
colnames(rf_result) <- c("ntree","ACC", "BCR")

In [7]:
RF_model1 <- randomForest(damage_grade ~ ., data = all_train, ntree = 30, importance = TRUE)

In [8]:
importance(RF_model1)[order(importance(RF_model1)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,32.73467,11383.6502
age,30.94528,4156.9538
geo_level_2_id,30.30032,6964.7216
geo_level_3_id,29.90210,6393.7331
height_percentage,26.10328,2775.9213
area_percentage,24.81629,4275.8822
has_superstructure_mud_mortar_brick,21.53827,475.5810
has_superstructure_rc_non_engineered,19.11217,346.8733
3,17.27661,646.1823
has_superstructure_cement_mortar_stone,17.26379,256.4426


In [9]:
importance(RF_model1)[order(importance(RF_model1)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,32.73467279,11383.650
geo_level_2_id,30.30032165,6964.722
geo_level_3_id,29.90209964,6393.733
building_id,0.09294729,6005.752
area_percentage,24.81629247,4275.882
age,30.94528252,4156.954
height_percentage,26.10327676,2775.921
6,10.66248166,1389.887
count_floors_pre_eq,16.01814772,1366.091
has_superstructure_mud_mortar_stone,15.92807824,1268.620


In [10]:
pred <- predict(RF_model1, test)
cm <- table(test$damage_grade,pred)
rf_result[1,1] <- 30
rf_result[1,2:3] <- perf_eval_multi(cm)

In [11]:
RF_model2 <- randomForest(damage_grade ~ ., data = all_train, ntree = 60, importance = TRUE)

In [12]:
importance(RF_model2)[order(importance(RF_model2)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_2_id,60.60564,7053.8848
geo_level_3_id,55.33501,6326.5697
geo_level_1_id,41.13225,10678.8428
area_percentage,38.01723,4178.1346
age,35.88104,4133.9596
height_percentage,28.38521,2801.2026
has_superstructure_stone_flag,27.53904,446.5387
has_superstructure_adobe_mud,25.68601,556.7702
has_superstructure_timber,24.13497,804.9308
count_floors_pre_eq,22.54631,1315.2292


In [13]:
importance(RF_model2)[order(importance(RF_model2)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,41.1322464,10678.843
geo_level_2_id,60.6056441,7053.885
geo_level_3_id,55.3350065,6326.570
building_id,0.3598482,5916.854
area_percentage,38.0172256,4178.135
age,35.8810392,4133.960
height_percentage,28.3852097,2801.203
6,14.8771677,1677.214
count_floors_pre_eq,22.5463096,1315.229
has_superstructure_mud_mortar_stone,19.5291748,1220.708


In [14]:
pred <- predict(RF_model2, test)
cm <- table(test$damage_grade,pred)
rf_result[2,1] <- 60
rf_result[2,2:3] <- perf_eval_multi(cm)

In [15]:
RF_model3 <- randomForest(damage_grade ~ ., data = all_train, ntree = 90, importance = TRUE)

In [16]:
importance(RF_model3)[order(importance(RF_model3)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_3_id,62.50566,6360.5222
geo_level_2_id,61.83157,6970.3771
geo_level_1_id,59.42665,10815.3842
age,39.37738,4153.9828
height_percentage,38.47259,2859.4437
area_percentage,36.59141,4188.0494
count_floors_pre_eq,33.84583,1313.5237
3,32.69437,633.9734
has_superstructure_timber,32.51548,862.1900
has_superstructure_stone_flag,32.03653,461.4869


In [17]:
importance(RF_model3)[order(importance(RF_model3)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,59.4266505,10815.384
geo_level_2_id,61.8315677,6970.377
geo_level_3_id,62.5056623,6360.522
building_id,0.2196931,5979.942
area_percentage,36.5914139,4188.049
age,39.3773808,4153.983
height_percentage,38.4725939,2859.444
6,16.8599819,1530.966
has_superstructure_mud_mortar_stone,24.1470154,1325.829
count_floors_pre_eq,33.8458349,1313.524


In [18]:
pred <- predict(RF_model3, test)
cm <- table(test$damage_grade,pred)
rf_result[3,1] <- 90
rf_result[3,2:3] <- perf_eval_multi(cm)

In [19]:
RF_model4 <- randomForest(damage_grade ~ ., data = all_train, ntree = 120, importance = TRUE)

In [20]:
importance(RF_model4)[order(importance(RF_model4)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_3_id,77.50807,6275.4595
geo_level_2_id,75.39719,7010.3391
geo_level_1_id,71.92578,10714.1116
area_percentage,54.07018,4191.0831
height_percentage,47.03389,2849.1494
age,39.52765,4101.8497
has_superstructure_adobe_mud,38.43454,566.8922
count_floors_pre_eq,37.87799,1277.8694
has_superstructure_stone_flag,34.15385,458.2168
has_superstructure_timber,33.84484,835.6177


In [21]:
importance(RF_model4)[order(importance(RF_model4)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,71.925781,10714.112
geo_level_2_id,75.397189,7010.339
geo_level_3_id,77.508074,6275.460
building_id,1.384627,5947.282
area_percentage,54.070175,4191.083
age,39.527648,4101.850
height_percentage,47.033891,2849.149
6,17.310123,1515.007
count_floors_pre_eq,37.877988,1277.869
count_families,22.023435,1217.892


In [22]:
pred <- predict(RF_model4, test)
cm <- table(test$damage_grade,pred)
rf_result[4,1] <- 120
rf_result[4,2:3] <- perf_eval_multi(cm)

In [23]:
RF_model5 <- randomForest(damage_grade ~ ., data = all_train, ntree = 150, importance = TRUE)

In [24]:
importance(RF_model5)[order(importance(RF_model5)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_3_id,79.00494,6370.8505
geo_level_2_id,77.53467,7040.6120
geo_level_1_id,75.30981,10949.3734
area_percentage,60.79212,4217.9979
height_percentage,54.54949,2860.6149
age,44.97294,4171.9670
has_superstructure_adobe_mud,41.37893,585.3485
has_superstructure_timber,41.09861,805.7397
3,39.67711,638.9922
1,37.72519,576.3503


In [25]:
importance(RF_model5)[order(importance(RF_model5)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,75.3098075,10949.373
geo_level_2_id,77.5346653,7040.612
geo_level_3_id,79.0049354,6370.851
building_id,0.3572611,5998.067
area_percentage,60.7921182,4217.998
age,44.9729397,4171.967
height_percentage,54.5494947,2860.615
6,21.1011220,1364.468
has_superstructure_mud_mortar_stone,30.5465995,1352.704
count_floors_pre_eq,37.5824800,1306.269


In [26]:
pred <- predict(RF_model5, test)
cm <- table(test$damage_grade,pred)
rf_result[5,1] <- 150
rf_result[5,2:3] <- perf_eval_multi(cm)

In [27]:
RF_model6 <- randomForest(damage_grade ~ ., data = all_train, ntree = 180, importance = TRUE)

In [28]:
importance(RF_model6)[order(importance(RF_model6)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_2_id,93.40652,6947.7164
geo_level_3_id,83.44628,6271.3084
geo_level_1_id,75.78440,10808.4562
area_percentage,66.24596,4207.0532
height_percentage,56.86605,2850.4187
age,53.80933,4079.9566
has_superstructure_timber,43.40285,836.7401
count_floors_pre_eq,42.83905,1304.5651
has_superstructure_adobe_mud,41.89307,594.3970
3,39.84965,629.0285


In [29]:
importance(RF_model6)[order(importance(RF_model6)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,75.78439909,10808.456
geo_level_2_id,93.40651640,6947.716
geo_level_3_id,83.44628311,6271.308
building_id,-0.03053057,5906.696
area_percentage,66.24596416,4207.053
age,53.80933439,4079.957
height_percentage,56.86604813,2850.419
6,23.97485242,1414.895
has_superstructure_mud_mortar_stone,31.97057048,1405.229
count_floors_pre_eq,42.83905226,1304.565


In [30]:
pred <- predict(RF_model6, test)
cm <- table(test$damage_grade,pred)
rf_result[6,1] <- 180
rf_result[6,2:3] <- perf_eval_multi(cm)

In [31]:
RF_model7 <- randomForest(damage_grade ~ ., data = all_train, ntree = 210, importance = TRUE)

In [32]:
importance(RF_model7)[order(importance(RF_model7)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_3_id,101.18302,6374.1560
geo_level_2_id,95.25848,7030.2878
geo_level_1_id,85.00256,10914.3754
area_percentage,63.95605,4226.7300
height_percentage,63.26053,2876.4582
age,61.26433,4151.8825
has_superstructure_adobe_mud,48.79679,588.1430
has_superstructure_timber,48.76528,836.5124
has_superstructure_stone_flag,48.22621,480.9988
3,47.75287,650.1448


In [33]:
importance(RF_model7)[order(importance(RF_model7)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,85.0025644,10914.375
geo_level_2_id,95.2584810,7030.288
geo_level_3_id,101.1830240,6374.156
building_id,0.2297114,6020.705
area_percentage,63.9560525,4226.730
age,61.2643273,4151.883
height_percentage,63.2605331,2876.458
6,26.7516074,1796.062
count_floors_pre_eq,44.9512701,1292.289
has_superstructure_mud_mortar_stone,34.2561149,1269.066


In [34]:
pred <- predict(RF_model7, test)
cm <- table(test$damage_grade,pred)
rf_result[7,1] <- 210
rf_result[7,2:3] <- perf_eval_multi(cm)

In [35]:
RF_model8 <- randomForest(damage_grade ~ ., data = all_train, ntree = 240, importance = TRUE)

In [36]:
importance(RF_model8)[order(importance(RF_model8)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_2_id,102.37152,6990.3074
geo_level_3_id,91.45475,6332.3985
geo_level_1_id,84.38659,10831.2942
area_percentage,73.71984,4216.3497
age,63.96292,4162.2864
height_percentage,63.87032,2837.4137
count_floors_pre_eq,50.10605,1289.8646
has_superstructure_adobe_mud,50.06545,589.6675
has_superstructure_timber,49.68090,854.3265
3,47.23244,637.6850


In [37]:
importance(RF_model8)[order(importance(RF_model8)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,84.3865915,10831.294
geo_level_2_id,102.3715161,6990.307
geo_level_3_id,91.4547534,6332.398
building_id,0.3589888,5953.415
area_percentage,73.7198432,4216.350
age,63.9629204,4162.286
height_percentage,63.8703222,2837.414
6,27.2929325,1508.317
has_superstructure_mud_mortar_stone,36.6054419,1356.355
count_floors_pre_eq,50.1060521,1289.865


In [38]:
pred <- predict(RF_model8, test)
cm <- table(test$damage_grade,pred)
rf_result[8,1] <- 240
rf_result[8,2:3] <- perf_eval_multi(cm)

In [39]:
RF_model9 <- randomForest(damage_grade ~ ., data = all_train, ntree = 270, importance = TRUE)

In [40]:
importance(RF_model9)[order(importance(RF_model9)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_2_id,110.13349,7016.1146
geo_level_3_id,98.65968,6340.1802
geo_level_1_id,96.88792,10829.7548
height_percentage,73.02269,2861.5747
area_percentage,72.63977,4197.2699
age,66.97961,4131.4341
has_superstructure_adobe_mud,56.07806,581.8898
has_superstructure_timber,54.47628,845.1002
3,52.26499,634.3857
count_floors_pre_eq,51.78611,1295.3844


In [41]:
importance(RF_model9)[order(importance(RF_model9)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,96.8879247,10829.755
geo_level_2_id,110.1334891,7016.115
geo_level_3_id,98.6596762,6340.180
building_id,-0.3398225,5947.657
area_percentage,72.6397724,4197.270
age,66.9796133,4131.434
height_percentage,73.0226931,2861.575
6,28.3873152,1526.175
count_floors_pre_eq,51.7861069,1295.384
has_superstructure_mud_mortar_stone,41.1914427,1239.941


In [42]:
pred <- predict(RF_model9, test)
cm <- table(test$damage_grade,pred)
rf_result[9,1] <- 270
rf_result[9,2:3] <- perf_eval_multi(cm)

In [43]:
RF_model10 <- randomForest(damage_grade ~ ., data = all_train, ntree = 300, importance = TRUE)

In [44]:
importance(RF_model10)[order(importance(RF_model10)[,4],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_2_id,119.41015,7021.6290
geo_level_3_id,105.24278,6351.5723
geo_level_1_id,101.62712,10977.3923
area_percentage,82.73428,4229.7060
height_percentage,78.26973,2882.0387
age,69.68032,4183.1613
has_superstructure_timber,58.11288,848.2388
count_floors_pre_eq,56.52828,1308.5686
3,55.83557,643.3361
has_superstructure_adobe_mud,55.01630,595.2561


In [45]:
importance(RF_model10)[order(importance(RF_model10)[,5],decreasing = TRUE),][1:10,4:5]

,MeanDecreaseAccuracy,MeanDecreaseGini
geo_level_1_id,101.62711789,10977.392
geo_level_2_id,119.41014939,7021.629
geo_level_3_id,105.24278432,6351.572
building_id,0.02973738,5970.989
area_percentage,82.73427750,4229.706
age,69.68032484,4183.161
height_percentage,78.26973442,2882.039
6,32.19877008,1519.460
has_superstructure_mud_mortar_stone,39.41097866,1333.587
count_floors_pre_eq,56.52827562,1308.569


In [46]:
pred <- predict(RF_model10, test)
cm <- table(test$damage_grade,pred)
rf_result[10,1] <- 300
rf_result[10,2:3] <- perf_eval_multi(cm)

In [47]:
rf_result

ntree,ACC,BCR
30,0.7072326,0.5766060
60,0.7116549,0.5748111
90,0.7104668,0.5744820
120,0.7121830,0.5776896
150,0.7132556,0.5797089
180,0.7122985,0.5763037
210,0.7129255,0.5775153
240,0.7134371,0.5775716
270,0.7136021,0.5787192
300,0.7139981,0.5780241
